<a href="https://colab.research.google.com/github/radr44/Iterative_Convolutional_Neural_Network/blob/master/trainingresults.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import torch
import numpy as np
import cv2 as cv
import subsampling
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import lr_scheduler
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import ICNN
import preprocessing
import copy

In [0]:
data_transforms = transforms.Compose([
        transforms.Resize((96,144)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor() ])

In [0]:
data_dir = '/content/drive/My Drive/Superset'
image_dataset = datasets.ImageFolder(data_dir,data_transforms)
class_names = image_dataset.classes
train_set, hold_set = torch.utils.data.random_split(image_dataset, [23582,10000])
train_set, throw_set = torch.utils.data.random_split(train_set, [23580,2] )
val_set, test_set = torch.utils.data.random_split(hold_set, [5000,5000])
dataset_train = torch.utils.data.DataLoader(train_set, batch_size = 5, shuffle = True, num_workers = 4)
dataset_val = torch.utils.data.DataLoader(val_set, batch_size = 5, shuffle = True, num_workers = 4)
dataset_test = torch.utils.data.DataLoader(test_set, batch_size = 5, shuffle = True, num_workers = 4)
class_names

In [0]:
def train_model_p1(model, criterion, optimizer, scheduler, num_epochs=10):
  #train whole network using the fc layers of last ucnn
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'dev']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for batch_idx,(inputs, labels) in enumerate(dataset_combined[phase]):
               
               
                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
                if phase== 'train':
                    if batch_idx % 50 == 0:
                        print('Epoch : {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}\t Accuracy:{:.3f}%'.format(
                      epoch, batch_idx*len(inputs), len(dataset_combined["train"].dataset), 100*batch_idx / len(dataset_train), loss.data, float(running_corrects*100) / float(5*(batch_idx+1))))
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'dev' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))
    print(scheduler)

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [0]:
dataset_combined = {"train":dataset_train,"dev":dataset_val}
dataset_sizes = {"train":len(train_set),"dev":len(val_set)}

In [0]:
ucnn_1 = ICNN.micro_cnn_Alexnet(k=0)
ucnn_2 = ICNN.micro_cnn_Alexnet(k=1)
ucnn_3 = ICNN.micro_cnn_Alexnet(k=2)
ucnn_4 = ICNN.micro_cnn_Alexnet(k=3)
ucnn_5 = ICNN.micro_cnn_Alexnet(k=4)
ucnn_6 = ICNN.micro_cnn_Alexnet(k=5)
ucnn_7 = ICNN.micro_cnn_Alexnet(k=6)
ucnn_8 = ICNN.micro_cnn_Alexnet(k=7)
ucnn_9 = ICNN.micro_cnn_Alexnet(k=8)
ucnn_10 = ICNN.micro_cnn_Alexnet(k=9,nfc=512)
ucnn = [ucnn_1,ucnn_2,ucnn_3,ucnn_4,ucnn_5,ucnn_6]# training for 6 ucnns

In [0]:
icnn=ICNN.ICNN_Alexnet(ucnn=ucnn,batch_size=5)
criterion = nn.CrossEntropyLoss()

optimizer_ft = optim.Adam(icnn.parameters(),lr=0.0001)
lmbda = lambda epoch: 0.95

mul_lr_scheduler = lr_scheduler.MultiplicativeLR(optimizer_ft,lr_lambda=lmbda)

In [8]:
len(list(icnn.parameters()))

96

In [0]:
model_ft = train_model_p1(icnn, criterion, optimizer_ft, mul_lr_scheduler,num_epochs=8)

Epoch 0/7
----------
Epoch : 0 [0/23580 (0%)]	Loss: 2.730670	 Accuracy:0.000%
Epoch : 0 [250/23580 (1%)]	Loss: 2.686708	 Accuracy:7.451%
Epoch : 0 [500/23580 (2%)]	Loss: 2.705427	 Accuracy:9.901%
Epoch : 0 [750/23580 (3%)]	Loss: 2.291590	 Accuracy:12.053%
Epoch : 0 [1000/23580 (4%)]	Loss: 2.184672	 Accuracy:17.015%
Epoch : 0 [1250/23580 (5%)]	Loss: 2.016990	 Accuracy:21.514%
Epoch : 0 [1500/23580 (6%)]	Loss: 1.366412	 Accuracy:25.714%
Epoch : 0 [1750/23580 (7%)]	Loss: 0.384138	 Accuracy:29.744%
Epoch : 0 [2000/23580 (8%)]	Loss: 1.373826	 Accuracy:32.968%
Epoch : 0 [2250/23580 (10%)]	Loss: 1.090856	 Accuracy:36.098%
Epoch : 0 [2500/23580 (11%)]	Loss: 1.014690	 Accuracy:39.521%
Epoch : 0 [2750/23580 (12%)]	Loss: 2.191034	 Accuracy:41.488%
Epoch : 0 [3000/23580 (13%)]	Loss: 0.788812	 Accuracy:43.594%
Epoch : 0 [3250/23580 (14%)]	Loss: 0.788342	 Accuracy:45.499%
Epoch : 0 [3500/23580 (15%)]	Loss: 1.304036	 Accuracy:47.104%
Epoch : 0 [3750/23580 (16%)]	Loss: 0.403041	 Accuracy:48.868%
Epoch

In [0]:
torch.save(model_ft,"/content/drive/My Drive/ICNN_weights.pt")

In [0]:
model_ft5 = torch.load("/content/drive/My Drive/ICNN_weights.pt")
model_ft5.eval()
model_ft5.n_max = 5

In [0]:
def count_parameters(model):
    i=0
    for name,param in model.named_parameters():
      if param.requires_grad == True:
       print(name)

In [11]:
#freezing the conv layers to train the fc layers of the microcnns withn<n_max
for child in model_ft5.ucnn.children():
    child.conv1b.weight.requires_grad = False
    child.conv2.weight.requires_grad = False
    child.conv3.weight.requires_grad = False
    child.conv4.weight.requires_grad = False
    child.conv5.weight.requires_grad = False
    child.conv1b.bias.requires_grad = False
    child.conv2.bias.requires_grad = False
    child.conv3.bias.requires_grad = False
    child.conv4.bias.requires_grad = False
    child.conv5.bias.requires_grad = False
count_parameters(model_ft5)    


ucnn.0.fc1.weight
ucnn.0.fc1.bias
ucnn.0.fc2.weight
ucnn.0.fc2.bias
ucnn.0.fc3.weight
ucnn.0.fc3.bias
ucnn.1.fc1.weight
ucnn.1.fc1.bias
ucnn.1.fc2.weight
ucnn.1.fc2.bias
ucnn.1.fc3.weight
ucnn.1.fc3.bias
ucnn.2.fc1.weight
ucnn.2.fc1.bias
ucnn.2.fc2.weight
ucnn.2.fc2.bias
ucnn.2.fc3.weight
ucnn.2.fc3.bias
ucnn.3.fc1.weight
ucnn.3.fc1.bias
ucnn.3.fc2.weight
ucnn.3.fc2.bias
ucnn.3.fc3.weight
ucnn.3.fc3.bias
ucnn.4.fc1.weight
ucnn.4.fc1.bias
ucnn.4.fc2.weight
ucnn.4.fc2.bias
ucnn.4.fc3.weight
ucnn.4.fc3.bias
ucnn.5.fc1.weight
ucnn.5.fc1.bias
ucnn.5.fc2.weight
ucnn.5.fc2.bias
ucnn.5.fc3.weight
ucnn.5.fc3.bias


In [13]:
#one epoch is sufficient to train the fc layers as the conv layers weights have been freezed
#using n_max = m will force the mth ucnn to use its fc layers for classification and hence would be trained 
#running for a few more epochs can give better accuracy
#note that as observed in the previous example the accuracy increases drastically after each epoch,this is because during the initial 
#classifications a relatively untrained model when compared to the one used in the end was being used which led to many misclassifications
#being accumulated which show up in the cummilative accuracy 
optimizer_ft5 = optim.Adam(model_ft5.parameters(),lr=0.0001)
#scheduler useless
mul_lr_scheduler_ft5 = lr_scheduler.MultiplicativeLR(optimizer_ft5,lr_lambda=lmbda)
model_ft5 = train_model_p1(model_ft5, criterion, optimizer_ft5, mul_lr_scheduler_ft5,num_epochs=1)

Epoch 0/0
----------
Epoch : 0 [0/23580 (0%)]	Loss: 3.291381	 Accuracy:20.000%
Epoch : 0 [250/23580 (1%)]	Loss: 1.071622	 Accuracy:47.451%
Epoch : 0 [500/23580 (2%)]	Loss: 0.675661	 Accuracy:60.990%
Epoch : 0 [750/23580 (3%)]	Loss: 0.579972	 Accuracy:67.815%
Epoch : 0 [1000/23580 (4%)]	Loss: 0.316203	 Accuracy:72.338%
Epoch : 0 [1250/23580 (5%)]	Loss: 0.482187	 Accuracy:74.900%
Epoch : 0 [1500/23580 (6%)]	Loss: 0.468850	 Accuracy:77.076%
Epoch : 0 [1750/23580 (7%)]	Loss: 0.346146	 Accuracy:78.860%
Epoch : 0 [2000/23580 (8%)]	Loss: 0.023910	 Accuracy:80.748%
Epoch : 0 [2250/23580 (10%)]	Loss: 0.339136	 Accuracy:82.129%
Epoch : 0 [2500/23580 (11%)]	Loss: 0.384652	 Accuracy:83.553%
Epoch : 0 [2750/23580 (12%)]	Loss: 0.055271	 Accuracy:84.356%
Epoch : 0 [3000/23580 (13%)]	Loss: 0.027452	 Accuracy:85.158%
Epoch : 0 [3250/23580 (14%)]	Loss: 0.085553	 Accuracy:85.899%
Epoch : 0 [3500/23580 (15%)]	Loss: 0.011938	 Accuracy:86.648%
Epoch : 0 [3750/23580 (16%)]	Loss: 0.013929	 Accuracy:87.244%
Ep

In [20]:
#train fc layers of 4th microcnn
optimizer_ft5 = optim.Adam(model_ft5.parameters(),lr=0.0001)
#scheduler useless
model_ft5.n_max=4
model_ft5 = train_model_p1(model_ft5, criterion, optimizer_ft5, mul_lr_scheduler,num_epochs=1)

Epoch 0/0
----------
Epoch : 0 [0/23580 (0%)]	Loss: 3.042996	 Accuracy:20.000%
Epoch : 0 [250/23580 (1%)]	Loss: 1.613456	 Accuracy:34.902%
Epoch : 0 [500/23580 (2%)]	Loss: 0.736864	 Accuracy:53.663%
Epoch : 0 [750/23580 (3%)]	Loss: 0.448069	 Accuracy:62.252%
Epoch : 0 [1000/23580 (4%)]	Loss: 0.464056	 Accuracy:68.557%
Epoch : 0 [1250/23580 (5%)]	Loss: 1.335544	 Accuracy:72.829%
Epoch : 0 [1500/23580 (6%)]	Loss: 0.078068	 Accuracy:75.548%
Epoch : 0 [1750/23580 (7%)]	Loss: 0.124822	 Accuracy:77.436%
Epoch : 0 [2000/23580 (8%)]	Loss: 0.077886	 Accuracy:79.152%
Epoch : 0 [2250/23580 (10%)]	Loss: 0.067825	 Accuracy:80.488%
Epoch : 0 [2500/23580 (11%)]	Loss: 0.251781	 Accuracy:81.836%
Epoch : 0 [2750/23580 (12%)]	Loss: 0.020359	 Accuracy:82.795%
Epoch : 0 [3000/23580 (13%)]	Loss: 0.419202	 Accuracy:83.594%
Epoch : 0 [3250/23580 (14%)]	Loss: 0.094805	 Accuracy:84.455%
Epoch : 0 [3500/23580 (15%)]	Loss: 0.020995	 Accuracy:85.050%
Epoch : 0 [3750/23580 (16%)]	Loss: 0.071593	 Accuracy:85.672%
Ep

In [21]:
#train fc layers of 3rd microcnn
optimizer_ft5 = optim.Adam(model_ft5.parameters(),lr=0.0001)
model_ft5.n_max=3
model_ft5 = train_model_p1(model_ft5, criterion, optimizer_ft5, mul_lr_scheduler,num_epochs=1)

Epoch 0/0
----------
Epoch : 0 [0/23580 (0%)]	Loss: 2.384450	 Accuracy:20.000%
Epoch : 0 [250/23580 (1%)]	Loss: 2.085176	 Accuracy:40.784%
Epoch : 0 [500/23580 (2%)]	Loss: 0.742033	 Accuracy:51.683%
Epoch : 0 [750/23580 (3%)]	Loss: 0.214412	 Accuracy:61.060%
Epoch : 0 [1000/23580 (4%)]	Loss: 0.071597	 Accuracy:67.761%
Epoch : 0 [1250/23580 (5%)]	Loss: 0.314706	 Accuracy:71.873%
Epoch : 0 [1500/23580 (6%)]	Loss: 0.235216	 Accuracy:74.485%
Epoch : 0 [1750/23580 (7%)]	Loss: 0.784312	 Accuracy:76.581%
Epoch : 0 [2000/23580 (8%)]	Loss: 0.175829	 Accuracy:78.304%
Epoch : 0 [2250/23580 (10%)]	Loss: 0.248390	 Accuracy:79.823%
Epoch : 0 [2500/23580 (11%)]	Loss: 0.010158	 Accuracy:81.238%
Epoch : 0 [2750/23580 (12%)]	Loss: 0.628791	 Accuracy:82.250%
Epoch : 0 [3000/23580 (13%)]	Loss: 0.037084	 Accuracy:83.095%
Epoch : 0 [3250/23580 (14%)]	Loss: 0.045490	 Accuracy:83.902%
Epoch : 0 [3500/23580 (15%)]	Loss: 0.230705	 Accuracy:84.593%
Epoch : 0 [3750/23580 (16%)]	Loss: 0.121809	 Accuracy:85.406%
Ep

In [22]:
#train fc layers of 2nd microcnn
optimizer_ft5 = optim.Adam(model_ft5.parameters(),lr=0.0001)
model_ft5.n_max=2
model_ft5 = train_model_p1(model_ft5, criterion, optimizer_ft5, mul_lr_scheduler,num_epochs=1)

Epoch 0/0
----------
Epoch : 0 [0/23580 (0%)]	Loss: 3.059305	 Accuracy:0.000%
Epoch : 0 [250/23580 (1%)]	Loss: 1.667382	 Accuracy:38.039%
Epoch : 0 [500/23580 (2%)]	Loss: 1.210273	 Accuracy:51.287%
Epoch : 0 [750/23580 (3%)]	Loss: 0.952789	 Accuracy:58.940%
Epoch : 0 [1000/23580 (4%)]	Loss: 1.459879	 Accuracy:62.687%
Epoch : 0 [1250/23580 (5%)]	Loss: 0.379074	 Accuracy:67.012%
Epoch : 0 [1500/23580 (6%)]	Loss: 1.139606	 Accuracy:70.233%
Epoch : 0 [1750/23580 (7%)]	Loss: 0.411074	 Accuracy:72.023%
Epoch : 0 [2000/23580 (8%)]	Loss: 0.420800	 Accuracy:74.065%
Epoch : 0 [2250/23580 (10%)]	Loss: 0.147286	 Accuracy:75.920%
Epoch : 0 [2500/23580 (11%)]	Loss: 0.434883	 Accuracy:77.126%
Epoch : 0 [2750/23580 (12%)]	Loss: 0.088872	 Accuracy:78.403%
Epoch : 0 [3000/23580 (13%)]	Loss: 0.485281	 Accuracy:79.368%
Epoch : 0 [3250/23580 (14%)]	Loss: 0.427932	 Accuracy:80.399%
Epoch : 0 [3500/23580 (15%)]	Loss: 0.547893	 Accuracy:81.369%
Epoch : 0 [3750/23580 (16%)]	Loss: 0.051056	 Accuracy:81.838%
Epo

In [23]:
#train fc layers of 1st microcnn
optimizer_ft5 = optim.Adam(model_ft5.parameters(),lr=0.0001)
model_ft5.n_max=1
model_ft5 = train_model_p1(model_ft5, criterion, optimizer_ft5, mul_lr_scheduler,num_epochs=1)

Epoch 0/0
----------
Epoch : 0 [0/23580 (0%)]	Loss: 2.646959	 Accuracy:20.000%
Epoch : 0 [250/23580 (1%)]	Loss: 2.609322	 Accuracy:19.608%
Epoch : 0 [500/23580 (2%)]	Loss: 2.431703	 Accuracy:27.327%
Epoch : 0 [750/23580 (3%)]	Loss: 1.598914	 Accuracy:32.980%
Epoch : 0 [1000/23580 (4%)]	Loss: 1.236975	 Accuracy:37.612%
Epoch : 0 [1250/23580 (5%)]	Loss: 1.264938	 Accuracy:41.833%
Epoch : 0 [1500/23580 (6%)]	Loss: 0.675446	 Accuracy:44.718%
Epoch : 0 [1750/23580 (7%)]	Loss: 0.964606	 Accuracy:47.806%
Epoch : 0 [2000/23580 (8%)]	Loss: 1.413279	 Accuracy:50.274%
Epoch : 0 [2250/23580 (10%)]	Loss: 1.138154	 Accuracy:52.284%
Epoch : 0 [2500/23580 (11%)]	Loss: 1.253709	 Accuracy:54.212%
Epoch : 0 [2750/23580 (12%)]	Loss: 0.473651	 Accuracy:55.681%
Epoch : 0 [3000/23580 (13%)]	Loss: 0.772941	 Accuracy:57.238%
Epoch : 0 [3250/23580 (14%)]	Loss: 0.934989	 Accuracy:58.802%
Epoch : 0 [3500/23580 (15%)]	Loss: 0.602865	 Accuracy:60.086%
Epoch : 0 [3750/23580 (16%)]	Loss: 0.133837	 Accuracy:61.225%
Ep

In [0]:
#we donot obtain much difference in training accuracies of the above iterations this can be attributed to a small dataset size
torch.save(model_ft5,"/content/drive/My Drive/ICNNfully_trained.pt")

In [0]:
from torch.autograd import Variable

In [0]:
def testing(model):
    correct = 0 
    for test_imgs, test_labels in dataset_test:
        #print(test_imgs.shape)
        test_imgs = Variable(test_imgs).float()
        output = model(test_imgs)
        predicted = torch.max(output,1)[1]
        correct += (predicted == test_labels).sum()
    print("Test accuracy:{:.3f}% ".format( 100*float(correct) / (len(dataset_test)*dataset_test.batch_size)))

In [27]:
#testing the model on test set for best case accuracy ie n_max =6, t_max = infinity
model_ft5.n_max=6
testing(model_ft5)

Test accuracy:99.120% 
